**Plan**

**1. Introduction to callbacks**

**2. Creating and using custom callback functions**

**3. Implementing custom layers in Keras models**

**4. Implementing custom models in Keras**






# **Introduction to callbacks**

Callbacks in Keras are a powerful feature that allows you to intervene during the training process of your model. They provide a way to execute certain actions at various stages of training, such as before or after each epoch or batch, or when certain conditions are met. This functionality can be used to monitor performance, adjust learning rates, save models, and more.

**<h2>Key Concepts</h2>**

1. **Callback Functions**:
   - Callbacks are functions or objects that are invoked during training at specific points, such as the beginning or end of an epoch, or after each batch.

2. **Custom Callbacks**:
   - Keras allows you to define custom callbacks by subclassing the `tf.keras.callbacks.Callback` class and overriding its methods.

**<h2>Common Built-in Callbacks</h2>**

1. **`ModelCheckpoint`**:
   - Saves the model or weights at regular intervals, typically after each epoch.
   - Useful for saving the best model based on validation performance.

   ```python
   from keras.callbacks import ModelCheckpoint

   checkpoint = ModelCheckpoint(
       'best_model.h5',            # File path to save the model
       monitor='val_loss',         # Monitor the validation loss
       save_best_only=True,         # Save only the best model
       mode='min',                 # Mode to determine the "best" model (min or max)
       verbose=1                   # Verbosity mode
   )
   ```

2. **`EarlyStopping`**:
   - Stops training when a monitored metric has stopped improving.
   - Helps prevent overfitting and unnecessary training.

   ```python
   from keras.callbacks import EarlyStopping

   early_stopping = EarlyStopping(
       monitor='val_loss',         # Monitor validation loss
       patience=10,                # Number of epochs with no improvement to wait
       mode='min',                 # Mode to determine the "best" model (min or max)
       verbose=1                   # Verbosity mode
   )
   ```

3. **`ReduceLROnPlateau`**:
   - Reduces the learning rate when a metric has stopped improving.
   - Useful for fine-tuning the learning rate.

   ```python
   from keras.callbacks import ReduceLROnPlateau

   reduce_lr = ReduceLROnPlateau(
       monitor='val_loss',         # Monitor validation loss
       factor=0.1,                 # Factor by which the learning rate will be reduced
       patience=5,                 # Number of epochs with no improvement to wait
       mode='min',                 # Mode to determine the "best" model (min or max)
       verbose=1                   # Verbosity mode
   )
   ```

4. **`TensorBoard`**:
   - Provides visualization of the training process, including metrics and model graph.
   - Integrates with TensorBoard, a tool for visualizing TensorFlow training runs.

   ```python
   from keras.callbacks import TensorBoard

   tensorboard = TensorBoard(
       log_dir='./logs',           # Directory where the logs will be saved
       histogram_freq=1,           # Frequency (in epochs) to compute activation and weight histograms
       write_graph=True,           # Whether to visualize the graph
       write_images=True           # Whether to write image summaries
   )
   ```

5. **`CSVLogger`**:
   - Streams training and validation metrics to a CSV file.
   - Useful for tracking and analyzing training history.

   ```python
   from keras.callbacks import CSVLogger

   csv_logger = CSVLogger(
       'training_log.csv',         # File path to save the CSV log
       append=True,                # Append to the existing file
       separator=',',              # Separator for the CSV file
       verbose=1                   # Verbosity mode
   )
   ```

**<h2>Creating Custom Callbacks</h2>**

You can create your own callbacks by subclassing `tf.keras.callbacks.Callback` and implementing the methods you need. Here’s an example of a custom callback that prints the current epoch number:

```python
from keras.callbacks import Callback

class PrintEpochCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1} has ended")

# Usage
print_epoch_callback = PrintEpochCallback()
```

**<h2>Using Callbacks During Training</h2>**

To use callbacks, pass them as a list to the `callbacks` parameter of the `fit` method:

```python
model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=50,
    batch_size=32,
    callbacks=[checkpoint, early_stopping, reduce_lr, tensorboard, csv_logger, print_epoch_callback]
)
```

**<h2>Summary</h2>**

Callbacks in Keras provide a flexible way to monitor and control the training process. Common built-in callbacks include `ModelCheckpoint`, `EarlyStopping`, `ReduceLROnPlateau`, `TensorBoard`, and `CSVLogger`. You can also create custom callbacks to implement specific functionalities. By leveraging callbacks, you can make your training process more efficient, automated, and insightful.

# **Creating and using custom callback functions**

Creating and using custom callback functions in Keras allows you to introduce tailored behaviors during training, such as custom logging, dynamic learning rate adjustments, or any other task that needs to be executed at specific points in the training process. Below, I’ll provide a step-by-step guide to creating and using custom callbacks in Keras.

**<h2>Step-by-Step Guide to Custom Callbacks</h2>**

**1. Subclass the `Callback` Class**

To create a custom callback, subclass `tf.keras.callbacks.Callback` and override the methods you are interested in. Common methods to override include:

- `on_epoch_begin(self, epoch, logs=None)`: Called at the beginning of each epoch.
- `on_epoch_end(self, epoch, logs=None)`: Called at the end of each epoch.
- `on_batch_begin(self, batch, logs=None)`: Called at the beginning of each batch.
- `on_batch_end(self, batch, logs=None)`: Called at the end of each batch.
- `on_train_begin(self, logs=None)`: Called at the beginning of training.
- `on_train_end(self, logs=None)`: Called at the end of training.



**Example 1: Custom Callback to Print Training Time**

Here’s an example of a custom callback that prints the time taken for each epoch:

In [34]:
import time
from keras.callbacks import Callback

class TimeHistory(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_duration = time.time() - self.epoch_start_time
        print(f"Epoch {epoch + 1} duration: {epoch_duration:.2f} seconds")

# Usage
time_callback = TimeHistory()

**Example 2: Custom Callback to Save Loss Values**

Here’s an example of a custom callback that saves the loss values at the end of each epoch to a list:

In [35]:
from keras.callbacks import Callback

class LossHistory(Callback):
    def on_train_begin(self, logs=None):
        self.losses = []

    def on_epoch_end(self, epoch, logs=None):
        self.losses.append(logs.get('loss'))

    def get_losses(self):
        return self.losses

# Usage
loss_history = LossHistory()

**Example 3: Custom Callback to Adjust Learning Rate**

Here’s an example of a custom callback that dynamically adjusts the learning rate based on the validation loss:

In [36]:
from keras.callbacks import Callback
import numpy as np

class LearningRateScheduler(Callback):
    def __init__(self, schedule):
        super().__init__()
        self.schedule = schedule

    def on_epoch_end(self, epoch, logs=None):
        current_lr = self.model.optimizer.learning_rate.numpy()
        new_lr = self.schedule(epoch, logs.get('val_loss'))
        if np.abs(new_lr - current_lr) > 1e-6:
            self.model.optimizer.learning_rate.assign(new_lr)
            print(f"Learning rate adjusted to: {new_lr:.6f}")

# Define the schedule function
def lr_schedule(epoch, val_loss):
    if epoch < 10:
        return 0.01
    elif epoch < 20:
        return 0.001
    else:
        return 0.0001

# Usage
lr_scheduler = LearningRateScheduler(lr_schedule)

**2. Integrate the Custom Callback into Model Training**

Once you have defined your custom callback, you can integrate it into the model training process by passing it to the callbacks argument of the fit method.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.datasets import mnist
from keras.utils import to_categorical

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Build a simple model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Create instances of the custom callbacks
time_callback = TimeHistory()
loss_history = LossHistory()
lr_scheduler = LearningRateScheduler(lr_schedule)

# Train the model
model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=30,
    batch_size=32,
    callbacks=[time_callback, loss_history, lr_scheduler]
)

# Access loss values from the custom callback
print("Training loss history:", loss_history.get_losses())

# **Implementing custom layers in Keras models**

Implementing custom layers in Keras models involves subclassing the tf.keras.layers.Layer class and defining the layer's functionality, including its forward pass, initialization, and (optionally) its configuration for saving and loading.

Custom layers can be useful when you need to create a new type of layer that isn't available in Keras, or when you want to experiment with novel layer designs.
Step-by-Step Guide to Implementing Custom Layers

**1. Subclass the tf.keras.layers.Layer Class**

To create a custom layer, you need to subclass tf.keras.layers.Layer and override several methods:

* __init__: Initialize the layer's parameters and any necessary configurations.
* build: Create the layer’s variables. This method is called once and should be used to create any weights or state variables.
* call: Define the computation that the layer performs. This method defines the forward pass of the layer.
* compute_output_shape: (Optional) Specify the output shape of the layer given an input shape.
* get_config: (Optional) Define how to serialize and deserialize the layer's configuration.



**Example 1: Custom Dense Layer**

Let’s create a custom dense layer that adds a bias to the output but does not use activation functions:

In [41]:
import keras

class CustomDense(keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(CustomDense, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        # Create trainable weights
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='random_normal',
            trainable=True
        )
        self.b = self.add_weight(
            shape=(self.units,),
            initializer='zeros',
            trainable=True
        )

    def call(self, inputs):
        return keras.ops.matmul(inputs, self.w) + self.b

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

    def get_config(self):
        config = super(CustomDense, self).get_config()
        config.update({"units": self.units})
        return config

**Example 2: Custom Activation Layer**

Now, let’s create a custom activation layer that applies a custom function to the inputs. For example, we'll implement a layer that applies a simple piecewise function:

In [42]:
class CustomActivation(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(CustomActivation, self).__init__(**kwargs)

    def call(self, inputs):
        # Apply custom piecewise activation function
        return keras.ops.where(inputs > 0, inputs, 0.1 * inputs)  # Leaky ReLU-like activation

    def get_config(self):
        return super(CustomActivation, self).get_config()


**2. Use Custom Layers in a Model**

Once you have defined your custom layers, you can use them in your Keras models like any other layer.

**Using CustomDense Layer:**

In [43]:
model = keras.Sequential([
    keras.layers.Input(shape=(784,)),
    CustomDense(128),  # Custom dense layer with 128 units
    keras.layers.ReLU(),
    CustomDense(10)    # Custom dense layer with 10 units
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ custom_dense (CustomDense)           │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu (ReLU)                         │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ custom_dense_1 (CustomDense)         │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

**Using CustomActivation Layer:**

In [47]:
model = keras.Sequential([
    keras.layers.Input(shape=(784,)),
    CustomDense(128),
    CustomActivation(),  # Custom activation layer
    CustomDense(10)
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ custom_dense_2 (CustomDense)         │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ custom_activation_1                  │ (None, 128)                 │               0 │
│ (CustomActivation)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ custom_dense_3 (CustomDense)         │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

**3. Saving and Loading Models with Custom Layers**

Custom layers need to be included in the model’s configuration to ensure they are properly saved and loaded.

**Saving the Model:**

In [48]:
model.save('model_with_custom_layers.keras')

**Loading the Model:**

To load a model with custom layers, you need to pass a custom_objects **dictionary to tf.keras.models.load_model:**

In [49]:
from keras.models import load_model

custom_objects = {
    'CustomDense': CustomDense,
    'CustomActivation': CustomActivation
}

model = load_model('model_with_custom_layers.keras', custom_objects=custom_objects)

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 10 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


# **Implementing custom models in Keras**

In [63]:
import keras
from keras.models import Model
from keras.layers import Dense, Input
from keras import backend as K
import tensorflow as tf

class CustomSequentialModel(Model):
    def __init__(self, units=64, **kwargs):
        super(CustomSequentialModel, self).__init__(**kwargs)
        # Define layers
        self.dense1 = Dense(units, activation='relu')
        self.dense2 = Dense(10, activation="softmax")  # Output layer with 10 units

    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return x

    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            # Forward pass
            y_pred = self(x, training=True)
            # Compute loss
            loss = self.compiled_loss(y, y_pred)  # Using compiled_loss for better compatibility

        # Compute gradients
        gradients = tape.gradient(loss, self.trainable_variables)
        # Apply gradients
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        # Update metrics
        for metric in self.metrics:
            metric.update_state(y, y_pred)
        metrics = {m.name: m.result() for m in self.metrics}

        return {**metrics, "loss": loss}

    def test_step(self, data):
        x, y = data

        # Forward pass
        y_pred = self(x, training=False)
        # Compute loss
        loss = self.compiled_loss(y, y_pred)  # Using compiled_loss for better compatibility

        # Update metrics
        for metric in self.metrics:
            metric.update_state(y, y_pred)
        metrics = {m.name: m.result() for m in self.metrics}

        return {**metrics, "loss": loss}

# Create and compile the model
model = CustomSequentialModel(units=128)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [61]:
model.summary()

Model: "custom_sequential_model_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_36 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [52]:
from keras.datasets import mnist
from keras.utils import to_categorical

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


In [64]:

# Fit the model
history = model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8756 - loss: 0.2582 - val_accuracy: 0.9601 - val_loss: 0.9293
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.9635 - loss: 0.1161 - val_accuracy: 0.9679 - val_loss: 0.9454
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9753 - loss: 0.0794 - val_accuracy: 0.9758 - val_loss: 0.9572
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9809 - loss: 0.0606 - val_accuracy: 0.9738 - val_loss: 0.9613
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9872 - loss: 0.0462 - val_accuracy: 0.9768 - val_loss: 0.9684
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9723 - loss: 0.0784
Test accuracy: 0.9768000245094299
